In [4]:
import sys
sys.path.append("..")

In [5]:
from textClassification import textAnalysis

In [158]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import nltk, re, string
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
import pandas as pd
from sklearn.metrics import accuracy_score

In [8]:
df = pd.read_csv("newtrain.csv", low_memory=False)

In [143]:
df_test = pd.read_csv("newtest.csv", low_memory=False)

In [9]:
df.head()

,Category,Text
0,5,why are yawns contagious? when people yawn
1,6,what is trans fat? how to reduce that? i heard...
2,1,roth ira vs 401k? what is the difference betwe...
3,1,how many planes fedex has? i heard that it is ...
4,2,what is the best photo slideshow creation appl...


In [15]:
allText = ""
for i in df.Text:
    allText += i + '.'

In [22]:
f = open('allText.txt', 'w')

In [23]:
f.write(allText)
f.close()

In [25]:
path = "allText.txt"

In [27]:
analysis = textAnalysis(path)

In [29]:
# tokenize sentences and words an run them through a POS tagger
sentences,tagged_sents = analysis.gen_corpus_prep()

# generate noun phrases and proper nouns
nps,nps_strings,fd_nps = analysis.chunker(sentences,tagged_sents)

# deduce synsets through lesk algorithm and calculate distance between synsets
final_list,hyponyms = analysis.lesk_comparison(nps,sentences)

# print final report
analysis.final_report(fd_nps,final_list,hyponyms)

Starting NP parsing.
Done with NP parsing.
Done with FD of NPs
Extracting nouns...
Done extracting nouns
Done running Lesk on nouns
Done creating FD on lemmas

Most frequent noun phrases:
---------------------------
.how do                 91   
br & gt                 27   
.what do                22   
my boyfriend            19   
a guy                   19   
my comput               16   
the world               15   
a girl                  14   
.whi do                 11   
.i am                   9    
.i want                 9    
my husband              9    


Most frequent lemmas from NPs and their hyponyms:
-------------------------------------------------
can             194      (containerful.n.01)
get             184      (return.n.11)
know            156      (knowing.n.01)
find            89       (act.n.02)
need            77       (psychological_feature.n.01)
want            70       (need.n.01)
people          58       (group.n.01)
someone         51       (organis

In [35]:
df.sort()

,Category,Text
0,5,why are yawns contagious? when people yawn
1,6,what is trans fat? how to reduce that? i heard...
2,1,roth ira vs 401k? what is the difference betwe...
3,1,how many planes fedex has? i heard that it is ...
4,2,what is the best photo slideshow creation appl...
5,3,what was the first dvd you ever bought? what w...
6,2,what is the singel most important thing you ar...
7,3,is lin qingxia (aka brigitte lin) the most bea...
8,5,who said the statement below and what does it ...
9,5,how do i find an out of print book? when i was...


In [82]:
cat_dict = dict()

In [83]:
cat_dict = {1:"Business&Finance",2:"Computers&Internet",3:"Entertainment&Music",4:"Family&Relationships",5:"Education&Reference",6:"Health",7:"Science&Mathematics"}

In [84]:
for i in range(7):    
    df_temp = df[df.Category == i + 1]
    size_temp = df_temp.shape[0]
    for r in range(size_temp):
        print("Category:",cat_dict[df_temp.irow(r).Category])
        print("Text:",df_temp.irow(r).Text)
        print("\n")

Category: Business&Finance
Text: roth ira vs 401k? what is the difference between roth ira and 401k? when should i prefer one over the other?


Category: Business&Finance
Text: how many planes fedex has? i heard that it is the largest airline in the world


Category: Business&Finance
Text: what are some tips on finding a good mortgage broker? what are some tips on finding a good mortgage broker?


Category: Business&Finance
Text: nice apartment building near sbc park? any recommendations for a nice apartment building in sf generally in the sbc park area?


Category: Business&Finance
Text: what is the best riddle that you know? i'm trying to have a library of the best riddles that people encountered. so a good riddle would be a riddle that has no ugly tricks in it- pure logic answer – no tricks


Category: Business&Finance
Text: economics of running a restaurant? running a restaurant looks like hard work and long hours. &#xd;&lt;br&gt;&lt;br&gt;what percentage of restaurants are profita

In [95]:
# creating feature for lengths of text and converting to pandas series
lengths = []
for i in df.Text.iteritems():
    lengths.append(len(i[1]))
lengths = pd.Series(lengths)

In [96]:
# appending lengths feature to dataframe
df['lengths'] = lengths

In [105]:
# aggregate all text
all_text = []
for i in df.Text:
    all_text.append(i)

In [125]:
for i in df.Text:
    print(i)
    print('\n')

why are yawns contagious? when people yawn


what is trans fat? how to reduce that? i heard that tras fat is bad for the body.  why is that? where can we find it in our daily food?


roth ira vs 401k? what is the difference between roth ira and 401k? when should i prefer one over the other?


how many planes fedex has? i heard that it is the largest airline in the world


what is the best photo slideshow creation application what is the best photo slideshow creation application


what was the first dvd you ever bought? what was the first dvd you ever bought?


what is the singel most important thing you are missing on the internet ? what is the single most important thing you are missing on the internet ?


is lin qingxia (aka brigitte lin) the most beautiful woman in chinese cinema? this is according to stephen chow (http://www.hkentreview.com/2005/features/kfh/kfhprem.html). is it true? who is the best-looking male star? did they make any movies together?


who said the statement bel

In [127]:
# aggregate all text
tok_sent = []
for i in df.Text:
    tok_sent.append(nltk.word_tokenize(i))

In [129]:
pos_sent = []
for i in tok_sent:
    pos_sent.append(nltk.pos_tag(i))

In [131]:
# counting nouns

noun_count = []
for q in pos_sent:
    counter = 0
    for w in q:
        if w[1] == 'NN':
            counter += 1
    noun_count.append(counter)

In [133]:
len(noun_count)

2698

In [135]:
# appending noun count feature to dataframe
df['nouns'] = noun_count

In [138]:
vec = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=5, max_features=1000)

In [146]:
# Train sparce vectorizing
arr_train_feature_sparse = vec.fit_transform(df.Text)

In [147]:
arr_train_feature = arr_train_feature_sparse.toarray()
arr_train_feature.shape

(2698, 1000)

In [150]:
# Test sparce vectorizing
arr_test_feature_sparse = vec.fit_transform(df_test.Text)

In [151]:
arr_test_feature = arr_test_feature_sparse.toarray()
arr_test_feature.shape

(1874, 1000)

In [142]:
# Trying logistic regression
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df.Category)

In [156]:
logreg_predictions = logreg_model.predict(arr_train_feature)

In [159]:
accuracy_score(df.Category, logreg_predictions)

0.7876204595997035

In [162]:
logreg_predictions = logreg_model.predict(arr_test_feature)

In [170]:
df_test['Prediction'] = pd.Series(logreg_predictions)

In [177]:
pd.DataFrame.to_csv(df_test[['Id','Prediction']], path_or_buf='submission_1.csv', header=['Id','Category'], index=False)